Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Build ML Pipeline with if-else node

In the [get started](../../../samples/basics//get-started.ipynb) tutorial we have introduced how to build an ML pipeline by using Azure Machine Learning components. In this tutorial, you will learn how to build pipeline with if-else node.



## Prerequisites
* Install private version of packages following the [instructions here](../README.md).

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Enable private features

In [8]:
# Import required libraries
import os


# enable private features
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "True"

## 1.2 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [9]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Shared token cache unavailable
	VisualStudioCodeCredential: Azure Active Directory error '(invalid_grant) AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2022-01-12T00:05:41.8429245Z and was inactive for 90.00:00:00.
Trace ID: 05ce40cd-303a-4274-8862-db5729442100
Correlation ID: ef745b05-2060-43eb-9d05-fbb7af77f6a5
Timestamp: 2022-10-25 03:30:02Z'
Content: {"error":"invalid_grant","error_description":"AADSTS700082: The refresh token has expired due to inactivity. The token was 

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/azureml-examples-v2/providers/Microsoft.MachineLearningServices/workspaces/main/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'd:\\azureml-examples\\sdk\\python\\jobs\\pipelines\\1j_pipeline_with_condition_flow\\if-else', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001E44D010580>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_D2_V2', 'min_instances': 0, 'max_instances': 40, 'idle_time_before_scale_down': 1800.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x000001E44CFFDB50>, 'tier': 'dedicated', 'subnet': None})


## 2. Build pipeline with if-else node
### 2.1 Create a component with conditional output

In [10]:
from component_with_boolean_output import condition_func

### 2.2 pipeline with if-else node

In [16]:
from mldesigner.dsl import condition # will merger to core sdk package
from azure.ai.ml import Input, load_component
from azure.ai.ml.dsl import pipeline


basic_component = load_component(
    source="helloworld_component.yaml"
)


@pipeline()
def condition_pipeline():
    result = condition_func()
    node1 = basic_component(component_in_number=1)
    node2 = basic_component(component_in_number=2)
    condition(condition=result.outputs.output, false_block=node1, true_block=node2)


pipeline_job = condition_pipeline()


# set pipeline level compute
pipeline_job.settings.default_compute = cluster_name

In [ ]:
# validate pipeline
ml_client.jobs.validate(pipeline_job)

### 2.3 Run pipeline job

In [ ]:
# pipeline parameter can be overridden when submit pipeline
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)

# show detail information of job
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)